# Originally adapted from IT_DEMO_MODEL_DEV

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

In [ ]:
%run ./model_end_events_for_outages_METHODS.ipynb

In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

#---------------------------------------------------------------------
import sys, os
import re
from pathlib import Path
import json
import pickle
import joblib

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version

import copy
import itertools
import adjustText

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from MeterPremise import MeterPremise
from EEMSP import EEMSP
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIEDE_DEV import AMIEDE_DEV
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
from OutagePredictor_TMPFAILSAFE import OutagePredictor
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, CombinedSQLWhereElements, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
#---------------------------------------------------------------------

In [ ]:
def plot_pred_probs_for_trsf_i(
    fig_num, 
    trsf_pole_nb_i, 
    y_prob1_by_date_df, 
    dovs_df, 
    trsf_pole_nb_col='trsf_pole_nb', 
    outg_rec_nb_col='OUTG_REC_NB', 
    dt_off_ts_full_col='DT_OFF_TS_FULL', 
    plot_maxima=False, 
    maxima_colors=['green', 'lawngreen']
):
    r"""
    dovs_df:
        Must be merged with MeterPremise so that it contains trsf_pole_nb_col
    """
    #-------------------------
    fig,ax = Plot_General.default_subplots(fig_num=fig_num)
    #-------------------------
    y_prob1_by_date_df.loc[trsf_pole_nb_i].T.plot.line(ax=ax)
    ax.axhline(0.5, color='cyan')
    #-------------------------
    dovs_df_i = dovs_df[dovs_df[trsf_pole_nb_col]==trsf_pole_nb_i][[outg_rec_nb_col, dt_off_ts_full_col]].drop_duplicates()
    if dovs_df_i.shape[0]>0:
        for idx_ij in range(dovs_df_i.shape[0]):
            dt_off_ts_full_ij = dovs_df_i.iloc[idx_ij][dt_off_ts_full_col]
            ax.axvline(dt_off_ts_full_ij, color='red', lw=5, alpha=0.5)
    ax.set_title(f'trsf_pole_nb = {trsf_pole_nb_i}')
    #-------------------------
    ax.set_ylim(0,1);
    #-------------------------
    if plot_maxima:
        idx_max_i = y_prob1_by_date_df.loc[trsf_pole_nb_i].idxmax()
        ax.axvline(idx_max_i, color=maxima_colors[0])
        if(
            idx_max_i > dovs_df_i[dt_off_ts_full_col].max() and
            y_prob1_by_date_df.loc[trsf_pole_nb_i].index[0] < dovs_df_i[dt_off_ts_full_col].max() #Make sure there are actually data to grab
        ):
            idx_max_i = y_prob1_by_date_df.loc[trsf_pole_nb_i][:dovs_df_i[dt_off_ts_full_col].max()].idxmax()
            ax.axvline(idx_max_i, color=maxima_colors[1])
    #-------------------------
    if dovs_df_i[dt_off_ts_full_col].min() < y_prob1_by_date_df.loc[trsf_pole_nb_i].index[0]:
        ax.set_xlim(left=dovs_df_i[dt_off_ts_full_col].min()-pd.Timedelta('1D'))    
    #-------------------------
    return fig,ax

In [ ]:
outg_rec_nbs_and_trsf_pole_nbs = [
    ('13276382', '41810769C10098'),
    ('13417866', '1892240742970'),
    ('13514788', '1913682718651'),
    ('13582234', '1894815709983'),
    ('13274721', '1889706782417'),
    ('13274721', '1892873787441'),
    ('13274721', '1888693793256'),
    ('13579067', '1890740711455'),
    ('13599801', '1839318753275'),
    ('13274721', '1892003777769'),
    ('13615261', '1876097823386'),
    ('13556889', '1879364759338'),
    ('13277281', '41810745C30405'),
    ('13417866', '1887212750550'),
    ('13463430', '1870422694331'),
    ('13578981', '1863880773804'),
    ('13277281', '41810770A40113'),
    ('13412351', '1847397697334'),
    ('13274721', '1891585782256'),
    ('13601025', '1838038754116'),
    ('13415340', '1913599716498'),
    ('13531769', '1909288704966'),
    ('13280212', '41810842A20014'),
    ('13275201', '40820707D30020'),
    ('13499294', '1857135747660'),
    ('13276753', '41810748C10029'),
    ('13417866', '1887498750606'),
    ('13274721', '1886814786624'),
    ('13274721', '1887172782079'),
    ('13542060', '1843907721312'),
    ('13277281', '41810769C30068'),
    ('13605911', '41810675D20034'),
    ('13276382', '41810745D30030'),
    ('13615261', '1880947815138'),
    ('13415340', '1912364714047'),
    ('13505364', '1881261732991'),
    ('13415340', '1913613713906'),
    ('13279407', '41810723B30030'),
    ('13278335', '41810797C20025'),
    ('13505053', '1832288678551'),
    ('13593685', '1856132744568'),
    ('13615589', '40820649C10174'),
    ('13566761', '1889994715166'),
    ('13600752', '41811006C40062'),
    ('13615261', '1873220814833'),
    ('13615261', '1883246815486'),
    ('13506106', '40830910D30055'),
    ('13282994', '41810748A20083'),
    ('13276382', '41810770A30083'),
    ('13274721', '1891692786587'),
    ('13615261', '1864852811557'),
    ('13285443', '41810745C40102'),
    ('13582234', '1894844709835'),
    ('13281046', '41810747B40067'),
    ('13274721', '1891784786477'),
    ('13514788', '1916013719745'),
    ('13280341', '41810703000066'),
    ('13516272', '1890110758410'),
    ('13601025', '1835993757440'),
    ('13278665', '40820506B10131'),
    ('13605678', '1896203737561'),
    ('13277281', '41810745C40067'),
    ('13612940', '1825272739214'),
    ('13417866', '1889973745283'),
    ('13282793', '41810723A40077'),
    ('13274721', '1889234786837'),
    ('13281046', '41810771A40219'),
    ('13277285', '40820484B40377'),
    ('13600752', '41811031A10117'),
    ('13276989', '40820281000067'),
    ('13279922', '41810774B40045'),
    ('13590801', '1904647698615'),
    ('13275201', '40820707C30174'),
    ('13516272', '1890317758297'),
    ('13590801', '1906962699848'),
    ('13275444', '40820530A10093'),
    ('13274721', '1893726785453'),
    ('13601025', '1843420754850'),
    ('13536002', '1957259749607'),
    ('13279922', '41810775B30039'),
    ('13499728', '1853486746307'),
    ('13564761', '40810165A20103'),
    ('13281137', '41810677B20026'),
    ('13276773', '41810770D10018'),
    ('13276382', '41810745D40419'),
    ('13277517', '41810771D40222'),
    ('13529152', '1861785484798'),
    ('13605911', '41810675D10007'),
    ('13605911', '41810675D20036'),
    ('13544724', '1894228701445'),
    ('13603418', '1865663759336'),
    ('13516272', '1889413761244'),
    ('13615261', '1875728824957'),
    ('13603418', '1864646759294'),
    ('13415340', '1912689713451'),
    ('13600752', '41811031A10267'),
    ('13582234', '1893664709718'),
    ('13579067', '1893078713191'),
    ('13276382', '41810769C40057'),
    ('13276382', '41810745D40366')
]

In [ ]:
outg_rec_nbs = [x[0] for x in outg_rec_nbs_and_trsf_pole_nbs]
dovs_dev = DOVSOutages(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True,
    build_sql_function=DOVSOutages_SQL.build_sql_std_outage, 
    build_sql_function_kwargs=dict(
        outg_rec_nbs=outg_rec_nbs, 
        field_to_split='outg_rec_nbs', 
        include_premise=False
    ), 
    build_consolidated=False
)
dovs_df_dev = dovs_dev.df.copy()

In [ ]:
dovs_df_dev

In [ ]:
# New meter_events.events_summary_vw table became live on 2023-04-01
# Therefore, we can only really assess outages occurring on/after 2023-05-02 (due to 31-day look)
outg_rec_nbs = dovs_df_dev[dovs_df_dev['DT_OFF_TS'] >= pd.to_datetime('2023-05-02')]['OUTG_REC_NB'].unique().tolist()
outg_rec_nbs_and_trsf_pole_nbs = [x for x in outg_rec_nbs_and_trsf_pole_nbs if x[0] in outg_rec_nbs]

In [ ]:
# prediction_dates = pd.date_range(
#     start = pd.to_datetime('2023-07-01'), 
#     end   = pd.to_datetime('2023-08-01'), 
#     freq  = pd.Timedelta('1D')
# )
# prediction_dates

prediction_dates = pd.date_range(
    start = pd.to_datetime('2023-05-02'), 
    end   = pd.to_datetime('2023-09-30'), 
    freq  = pd.Timedelta('1D')
)
prediction_dates

In [ ]:
%run ./model_end_events_for_outages_METHODS.ipynb

In [ ]:
start = time.time()

outg_prdctr = OutagePredictor(prediction_date=prediction_dates[0])
#-----
outg_prdctr.set_model_dir(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20231201\Models\All_EEMSP_agg_Top10_v2')
#-----
trsf_pole_nbs = [x[1] for x in outg_rec_nbs_and_trsf_pole_nbs]
outg_prdctr.set_trsf_pole_nbs(trsf_pole_nbs)
# outg_prdctr.set_trsf_pole_nbs_from_sql(
#     n_trsf_pole_nbs=10000, 
#     states='OH'
# )
# print(f"n_trsf_pole_nbs = {len(outg_prdctr.trsf_pole_nbs)}")
# print(f"\nSQL statement:\n{outg_prdctr.trsf_pole_nbs_sql}")
#-----
outg_prdctr.initialize_data(
    evsSum_sql_fcn=AMIEndEvents_SQL.build_sql_end_events,  
    evsSum_sql_kwargs=dict(opco='oh'), 
)


results_dict = dict()
for prediction_date_i in prediction_dates:
    print(prediction_date_i)
    assert(prediction_date_i not in results_dict)
    outg_prdctr.change_prediction_date(prediction_date=prediction_date_i)
    y_pred_i   = outg_prdctr.model_clf.predict(outg_prdctr.X_test)
    y_scores_i = outg_prdctr.model_clf.predict_proba(outg_prdctr.X_test)
    #-----
    assert(outg_prdctr.rcpx_df.shape[0]==len(y_pred_i))
    assert(outg_prdctr.rcpx_df.shape[0]==len(y_scores_i))
    res_i = pd.DataFrame(
        index=outg_prdctr.rcpx_df.index, 
        data=dict(
            y_pred=y_pred_i, 
            y_prob_0=[x[0] for x in y_scores_i], 
            y_prob_1=[x[1] for x in y_scores_i]
        )
    )    
    #-----
    results_dict[prediction_date_i] = res_i
    
print(time.time()-start)

In [ ]:
assert(0)